In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os
import sys
import torch

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
sys.path.append('..')

from cifar100cnn.data import get_cifar_data
from cifar100cnn.extractor import *

In [52]:
MODEL_PATHS = {
    "resnet18_scratch": "../checkpoints/resnet18/from-scratch/best_model.pth",
    "resnet50_scratch": "../checkpoints/resnet50/from-scratch/best_model.pth",
    "resnet18_fine_tuned": "../checkpoints/resnet18/fine-tuned/best_model.pth",
    "resnet50_fine_tuned": "../checkpoints/resnet50/fine-tuned/best_model.pth",
    "wide_resnet": "../checkpoints/wide_resnet28_10/best_model.pth"
}

In [53]:
def load_model(model_name, path, device):
    sys.path.append('..')
    
    if not os.path.exists(path):
        raise FileNotFoundError(f"Plik {path} nie istnieje")

    if "resnet18" in model_name:
        model = ResNet(version=18, num_classes=50, pretrained=False)
    elif "resnet50" in model_name:
        model = ResNet(version=50, num_classes=50, pretrained=False)
    elif "wide_resnet" in model_name:
        model = WideResNet(depth=28, widen_factor=10, dropout_rate=0.5, num_classes=50)
    else:
        raise ValueError(f"Nieznany model: {model_name}")

    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)

    model.to(device)
    model.eval()

    return model

In [54]:
train_loader, val_loader, test_loader, class_names = get_cifar_data(num_classes=100, augment=True)
class_dict = {idx: name for idx, name in enumerate(class_names)}

class_dict

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Saved class names to class_names.txt


{0: 'apple',
 1: 'aquarium_fish',
 2: 'baby',
 3: 'bear',
 4: 'beaver',
 5: 'bed',
 6: 'bee',
 7: 'beetle',
 8: 'bicycle',
 9: 'bottle',
 10: 'bowl',
 11: 'boy',
 12: 'bridge',
 13: 'bus',
 14: 'butterfly',
 15: 'camel',
 16: 'can',
 17: 'castle',
 18: 'caterpillar',
 19: 'cattle',
 20: 'chair',
 21: 'chimpanzee',
 22: 'clock',
 23: 'cloud',
 24: 'cockroach',
 25: 'couch',
 26: 'crab',
 27: 'crocodile',
 28: 'cup',
 29: 'dinosaur',
 30: 'dolphin',
 31: 'elephant',
 32: 'flatfish',
 33: 'forest',
 34: 'fox',
 35: 'girl',
 36: 'hamster',
 37: 'house',
 38: 'kangaroo',
 39: 'keyboard',
 40: 'lamp',
 41: 'lawn_mower',
 42: 'leopard',
 43: 'lion',
 44: 'lizard',
 45: 'lobster',
 46: 'man',
 47: 'maple_tree',
 48: 'motorcycle',
 49: 'mountain',
 50: 'mouse',
 51: 'mushroom',
 52: 'oak_tree',
 53: 'orange',
 54: 'orchid',
 55: 'otter',
 56: 'palm_tree',
 57: 'pear',
 58: 'pickup_truck',
 59: 'pine_tree',
 60: 'plain',
 61: 'plate',
 62: 'poppy',
 63: 'porcupine',
 64: 'possum',
 65: 'rabbit',

In [55]:
classes_A = np.arange(50)
classes_B = np.arange(50, 100)

models = {name: load_model(name, path, device) for name, path in MODEL_PATHS.items()}
feature_extractor = FeatureExtractor(device)

C:\Users\jedno\AppData\Local\Temp\ipykernel_14348\3110914763.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=device)


In [60]:
def calculate_accuracy(model, representative, test_loader, class_type):
    test_features, test_labels = feature_extractor.extract_features(model, test_loader)

    mask_test = np.isin(test_labels, class_type)
    test_features, test_labels_A = test_features[mask_test], test_labels[mask_test]

    predictions = classify_knn(test_features, representative)
    accuracy = evaluate_accuracy(predictions, test_labels_A)

    return accuracy

In [ ]:
def plot_stages_side_by_side(pca_results_A, pca_results_B, classes_list_A, classes_list_B, 
                            accuracies_A, accuracies_B, model_name, class_dict):
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))
    sample_counts = [1, 5, 10]

    # Etap A (dane treningowe)
    for col, num_samples in enumerate(sample_counts):
        idx = sample_counts.index(num_samples)
        features_pca = pca_results_A[idx]
        classes = classes_list_A[idx]
        accuracy = accuracies_A[idx]
        
        scatter = axs[0, col].scatter(features_pca[:, 0], features_pca[:, 1], c=classes, cmap='tab20', s=100, alpha=0.8)
        axs[0, col].set_title(f"Etap A ({num_samples} próbek/klasę)\nAccuracy: {accuracy:.2f}% (kNN cosine)")
        axs[0, col].set_xticks([])
        axs[0, col].set_yticks([])

        # Dodawanie nazw klas do wykresu
        for i, txt in enumerate(classes):
            axs[0, col].text(features_pca[i, 0], features_pca[i, 1], class_dict[txt], fontsize=9)

    # Etap B (dane testowe)
    for col, num_samples in enumerate(sample_counts):
        idx = sample_counts.index(num_samples)
        features_pca = pca_results_B[idx]
        classes = classes_list_B[idx]
        accuracy = accuracies_B[idx]
        
        scatter = axs[1, col].scatter(features_pca[:, 0], features_pca[:, 1], c=classes, cmap='tab20', s=100, alpha=0.8)
        axs[1, col].set_title(f"Etap B ({num_samples} próbek/klasę)\nAccuracy: {accuracy:.2f}% (kNN cosine)")
        axs[1, col].set_xticks([])
        axs[1, col].set_yticks([])

        # Dodawanie nazw klas do wykresu
        for i, txt in enumerate(classes):
            axs[1, col].text(features_pca[i, 0], features_pca[i, 1], class_dict[txt], fontsize=9)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.suptitle(f"Model: {model_name}", fontsize=16)
    plt.show()


# Ścieżki do folderów z danymi
base_dir_A = os.path.join("..", "etap2")
base_dir_B = os.path.join("..", "etap3")

models_names = ["resnet18_scratch", "resnet18_fine_tuned", "resnet50_scratch", "resnet50_fine_tuned", "wide_resnet"]
nums_samples = [1, 5, 10]

for model_name in models_names:
    # Inicjalizacja struktur danych
    pca_results_A, pca_results_B = [], []
    classes_list_A, classes_list_B = [], []
    accuracies_A, accuracies_B = [], []

    # Przetwarzanie Etapu A
    for num_samples in nums_samples:
        file_path = os.path.join(base_dir_A, f"{model_name}_{num_samples}_rep.npy")
        if not os.path.exists(file_path):
            continue

        loaded_data = np.load(file_path, allow_pickle=True).item()  # Dodaj .item()!

        # Odtworzenie struktury słownika
        class_representatives = {int(k): v for k, v in loaded_data.items()}

        classes = list(class_representatives.keys())
        features = np.array(list(class_representatives.values()))



        # Redukcja wymiarowości za pomocą PCA
        pca = PCA(n_components=2)
        features_pca = pca.fit_transform(features)
        
        # Dodanie wyników do list
        pca_results_A.append(features_pca)
        classes_list_A.append(classes)
        
        # Obliczanie accuracy dla Etapu A
        accuracy_A = calculate_accuracy(models[model_name], class_representatives, test_loader, classes_A)
        accuracies_A.append(accuracy_A)

    # Przetwarzanie Etapu B
    for num_samples in nums_samples:
        file_path = os.path.join(base_dir_B, f"{model_name}_{num_samples}.npy")
        if not os.path.exists(file_path):
            continue

        loaded_data = np.load(file_path, allow_pickle=True).item()  # Dodaj .item()!

        # Odtworzenie struktury słownika
        class_representatives = {int(k): v for k, v in loaded_data.items()}

        classes = list(class_representatives.keys())
        features = np.array(list(class_representatives.values()))


        # Redukcja wymiarowości za pomocą PCA
        pca = PCA(n_components=2)
        features_pca = pca.fit_transform(features)
        
        # Dodanie wyników do list
        pca_results_B.append(features_pca)
        classes_list_B.append(classes)
        
        # Obliczanie accuracy dla Etapu B
        accuracy_B = calculate_accuracy(models[model_name], class_representatives, test_loader, classes_B)
        accuracies_B.append(accuracy_B)

    # Generowanie wykresów
    if pca_results_A and pca_results_B:
        plot_stages_side_by_side(pca_results_A, pca_results_B, 
                                classes_list_A, classes_list_B, 
                                accuracies_A, accuracies_B, 
                                model_name, class_dict)

ValueError: can only convert an array of size 1 to a Python scalar